In [ ]:
# to chcieliśmy osiągnąć 
dziecko = Perceptron()
dziecko.fit(X,y)

In [1]:
# bo 

# obiektowosc sklearn
import numpy as np
import pandas as pd

dane = {'zm1':[3,4,np.NaN,2,6,1], 
        'zm2':[54,np.NaN, 56,342,643,123],
        'zm3':['niebieski','czerwony','czerwony',
               'niebieski','zielony','zielony'],
        'zm4':['male','male','female','male','female','female'],
        'target':[0,1,1,0,0,1]}

In [2]:
df = pd.DataFrame(dane)
df

,zm1,zm2,zm3,zm4,target
0,3.0,54.0,niebieski,male,0
1,4.0,NaN,czerwony,male,1
2,NaN,56.0,czerwony,female,1
3,2.0,342.0,niebieski,male,0
4,6.0,643.0,zielony,female,0
5,1.0,123.0,zielony,female,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   zm1     5 non-null      float64
 1   zm2     5 non-null      float64
 2   zm3     6 non-null      object 
 3   zm4     6 non-null      object 
 4   target  6 non-null      int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 368.0+ bytes


In [4]:
X = df.drop(columns=['target'])
y = df['target']

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [7]:
numeric_features = ["zm1", "zm2"]
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])
categorical_features = ["zm3", "zm4"]
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

In [8]:
preprocessor = ColumnTransformer(transformers=[
    ("num_trans", numeric_transformer, numeric_features),
    ("cat_trans", categorical_transformer, categorical_features)
])

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [10]:
pipeline = Pipeline(steps=[
    ("preproc", preprocessor),
    ("model", LogisticRegression())
])

In [11]:
from sklearn import set_config
set_config(display='diagram')
pipeline

Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('num_trans',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['zm1', 'zm2']),
                                                 ('cat_trans',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['zm3', 'zm4'])])),
                ('model', LogisticRegression())])

In [12]:
from sklearn.model_selection import train_test_split
X_tr, X_test, y_tr, y_test = train_test_split(X,y,
test_size=0.2, random_state=42)

In [13]:
pipeline.fit(X_tr, y_tr)

score = pipeline.score(X_test, y_test)
print(score)

0.5


In [14]:
import joblib
joblib.dump(pipeline, 'your_pipeline.pkl')

['your_pipeline.pkl']

In [15]:
param_grid = [
              {"preproc__num_trans__imputer__strategy":
              ["mean","median"],
               "model__n_estimators":[2,5,10,100,500],
               "model__min_samples_leaf": [1, 0.1],
               "model":[RandomForestClassifier()]},
              {"preproc__num_trans__imputer__strategy":
                ["mean","median"],
               "model__C":[0.1,1.0,10.0,100.0,1000],
                "model":[LogisticRegression()]}
]

from sklearn.model_selection import GridSearchCV


grid_search = GridSearchCV(pipeline, param_grid,
cv=2, verbose=1, n_jobs=-1)


grid_search.fit(X_tr, y_tr)

Fitting 2 folds for each of 30 candidates, totalling 60 fits


GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('preproc',
                                        ColumnTransformer(transformers=[('num_trans',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['zm1',
                                                                          'zm2']),
                                                                        ('cat_trans',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['zm3',
                                                                          'zm4'])])),
                                       ('model', LogisticRegression())]),
             n_jobs=-1,
             param_grid=[{'model': [RandomForestClassifier(n_estimators=2)],
                          'model__min_samples_leaf': [1, 0.1],
                          'model__n_estimators': [2, 5, 10, 100, 500],
                          'preproc__num_trans__imputer__strategy': ['mean',
                                                                    'median']},
                         {'model': [LogisticRegression()],
                          'model__C': [0.1, 1.0, 10.0, 100.0, 1000],
                          'preproc__num_trans__imputer__strategy': ['mean',
                                                                    'median']}],
             verbose=1)

In [16]:
grid_search.best_params_

{'model': RandomForestClassifier(n_estimators=2),
 'model__min_samples_leaf': 1,
 'model__n_estimators': 2,
 'preproc__num_trans__imputer__strategy': 'median'}

In [17]:
from sklearn.base import BaseEstimator, TransformerMixin

class DelOneValueFeature(BaseEstimator, TransformerMixin):
    """Transformacja usuwająca zmienne, które posiadają
    tylko jedną wartość w całej kolumnie. Takie kolumny
    nie nadają się do modelowania. Metoda fit() wyszuka
    wszystkie takie kolumny. Natomiast metoda transform()
    usunie je ze zbioru danych.
"""
    def __init__(self):
        self.one_value_features = []
    def fit(self, X, y=None):
        for feature in X.columns:
            unikalne = X[feature].unique()
            if len(unikalne)==1:
                self.one_value_features.append(feature)
        return self
    def transform(self, X, y=None):
        if not self.one_value_features:
            return X
        return X.drop(axis='columns',
        columns=self.one_value_features)
  

In [18]:
pipeline2 = Pipeline([
    ("moja_transformacja",DelOneValueFeature()),
    ("preprocesser", preprocessor),
    ("classifier", LogisticRegression())])
    
pipeline2.fit(X_tr, y_tr)
score2 = pipeline2.score(X_test, y_test)

In [19]:
score2

0.5

In [20]:
pipeline2

Pipeline(steps=[('moja_transformacja', DelOneValueFeature()),
                ('preprocesser',
                 ColumnTransformer(transformers=[('num_trans',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['zm1', 'zm2']),
                                                 ('cat_trans',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['zm3', 'zm4'])])),
                ('classifier', LogisticRegression())])

In [ ]:
class Perceptron():
    
    def __init__(self, eta=0.01, n_iter=10):
        self.eta = eta
        self.n_iter = n_iter
    
    def fit(self,X,y):
        self.w_ = np.zeros(1+X.shape[1])
        self.errors_ = []
        for _ in range(self.n_iter):
            pass
        return self

In [22]:
# własny model 

# implementacja 
class Perceptron():
    
    def __init__(self, eta=0.01, n_iter=10):
        self.eta = eta
        self.n_iter = n_iter
    
    def fit(self, X, y):
        self.w_ = np.zeros(1+X.shape[1])
        self.errors_ = []
        
        for _ in range(self.n_iter):
            errors = 0
            for xi, target in zip(X,y):
                #print(xi, target)
                update = self.eta*(target-self.predict(xi))
                #print(update)
                self.w_[1:] += update*xi
                self.w_[0] += update
                #print(self.w_)
                errors += int(update != 0.0)
            self.errors_.append(errors)
        return self
    
    def net_input(self, X):
        return np.dot(X, self.w_[1:]) + self.w_[0]
    
    def predict(self, X):
        return np.where(self.net_input(X)>=0.0,1,-1)

In [23]:
X=np.array([[1],[2],[3],[4]])
y=[-1,-1,1,1]

# AND
# X = np.array([0,0],[0,1],[1,0],[1,1])
# y = [-1,-1,-1,1]
# XOR

In [24]:
d = Perceptron()
d.fit(X,y)

print(d.errors_)
print(d.w_)

d.predict(np.array([-3]))
np.array([1,2,3,4]).reshape(-1,1)

d.predict(np.array([1,2,3,4,6,-1,23,-3]).reshape(-1,1))

[2, 3, 1, 0, 0, 0, 0, 0, 0, 0]
[-0.04  0.02]


array([-1, -1,  1,  1,  1, -1,  1, -1])

In [ ]:
# ZADANIE - Opisz czym różni się poniższy algorytm od Perceprtona ? 
class Adaline():
    '''Klasyfikator  - ADAptacyjny LIniowy NEuron'''
    def __init__(self, eta=0.01, n_iter=10):
        self.eta = eta
        self.n_iter = n_iter

    def fit(self, X,y):
        self.w_ = np.zeros(1+X.shape[1])
        self.cost_ = []

        for i in range(self.n_iter):
            net_input = self.net_input(X)
            output = self.activation(X)
            errors = (y-output)
            self.w_[1:] += self.eta * X.T.dot(errors)
            self.w_[0] += self.eta * errors.sum()
            cost = (errors**2).sum() / 2.0
            self.cost_.append(cost)
        return self

    def net_input(self, X):
        return np.dot(X, self.w_[1:]) + self.w_[0]

    def activation(self, X):
        return self.net_input(X)

    def predict(self, X):
        return np.where(self.activation(X) >= 0.0, 1, -1) 

In [ ]:
%%file app.py

import pickle
from math import log10

from flask import Flask
from flask import request
from flask import jsonify
import numpy as np

class Perceptron():
    
    def __init__(self,eta=0.01, n_iter=10):
        self.eta = eta
        self.n_iter = n_iter
    
    def fit(self, X, y):
        self.w_ = np.zeros(1+X.shape[1])
        self.errors_ = []
        
        for _ in range(self.n_iter):
            errors = 0
            for xi, target in zip(X,y):
                #print(xi, target)
                update = self.eta*(target-self.predict(xi))
                #print(update)
                self.w_[1:] += update*xi
                self.w_[0] += update
                #print(self.w_)
                errors += int(update != 0.0)
            self.errors_.append(errors)
        return self
    
    def net_input(self, X):
        return np.dot(X, self.w_[1:])+self.w_[0]
    
    def predict(self, X):
        return np.where(self.net_input(X)>=0.0,1,-1)

# Create a flask
app = Flask(__name__)

# Create an API end point
@app.route('/api/v1.0/predict', methods=['GET'])
def get_prediction():

    # sepal length
    sepal_length = float(request.args.get('sl'))
    # sepal width
    #sepal_width = float(request.args.get('sw'))
    # petal length
    petal_length = float(request.args.get('pl'))
    # petal width
    #petal_width = float(request.args.get('pw'))

    # The features of the observation to predict
    #features = [sepal_length,
    #            sepal_width,
    #            petal_length,
    #           petal_width]
    
    features = [sepal_length,
                petal_length]
    
    print(features)
    # Load pickled model file
    with open('model.pkl',"rb") as picklefile:
        model = pickle.load(picklefile)
    print(model)
    # Predict the class using the model
    predicted_class = int(model.predict(features))
    
    # Return a json object containing the features and prediction
    return jsonify(features=features, predicted_class=predicted_class)

if __name__ == '__main__':
    app.run()

In [ ]:
!python app.py

In [ ]:
# http://127.0.0.1:5000/api/v1.0/predict?&sl=4.5&pl=1.3

# pamiętaj otworzyć nowy notebook !
import requests
response = requests.get("http://127.0.0.1:5000/api/v1.0/predict?&sl=4.5&pl=1.3")
print(response.content)

In [25]:
from flask import Flask, request
from flask_restful import Resource, Api

from flask_jsonpify import jsonify

In [27]:
app  = Flask(__name__)

api = Api(app)

@app.route('/')
@app.route('/index')
def home():
    #return render_template('home.html')
    return "<h1>Strona poczatkowa</h1>"

@app.route('/hello/<name>')
def success(name):
    return f'<h2>{name}</h2>'

In [29]:
class Main(Resource):
    def get(self):
        return jsonify("Hello world")

In [30]:
api.add_resource(Main,'/test')

In [31]:
class Irys(Resource):
    
    def get(self):
        conn=engine.connect()
        query = conn.execute('select * from dane')
        result = {'dane':[i for i in query.cursor.fetchall()]}
        return jsonify(result)

In [32]:
api.add_resource(Irys, '/irys')

In [34]:
app.run(port='5002')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5002/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Nov/2021 12:56:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2021 12:57:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2021 12:57:02] "GET /test HTTP/1.1" 200 -


#  Python for Data Science

1. Podstawowe obiekty pythona dla przechowywania danych strukturyzowanych
    - listy 
    - tablice numpy
    - wielowymiarowe dane sklearn
    - dane generowane: make_classification()
    - DataFrame, podsatawowe operacje: `head()`, `tail()`, `info()`,`describe()`, `drop()`,`iloc[]`,`np.where()`
    - graficzna prezentacja danych `matplotlib`
    - SQLAlchemy funkcyjnie
    
2. Wczytywanie danych nieustrukturyzowanych 
    - zdjęcia: mnist, cifar10
    
3. Zapisanie obiektów z biblioteką pickle

4. Budowanie klas: 
    - Losowanie kości
    - Błądzenie losowe
    - Perceptron

In [35]:
# dane z internetu
import requests

res = requests.get('https://sebastianzajac.pl')

In [36]:
print(res)

<Response [200]>


In [37]:
res.headers

{'Connection': 'Keep-Alive', 'Keep-Alive': 'timeout=5, max=100', 'content-type': 'text/html; charset=UTF-8', 'transfer-encoding': 'chunked', 'content-encoding': 'gzip', 'vary': 'Accept-Encoding,User-Agent', 'date': 'Mon, 29 Nov 2021 11:58:36 GMT', 'server': 'LiteSpeed', 'alt-svc': 'h3=":443"; ma=2592000, h3-29=":443"; ma=2592000, h3-Q050=":443"; ma=2592000, h3-Q046=":443"; ma=2592000, h3-Q043=":443"; ma=2592000, quic=":443"; ma=2592000; v="43,46"'}

In [38]:
print(res.text)

<!DOCTYPE html>
<!--[if lt IE 8 ]><html class="no-js ie ie7" lang="en"> <![endif]-->
<!--[if IE 8 ]><html class="no-js ie ie8" lang="en"> <![endif]-->
<!--[if (gte IE 8)|!(IE)]><!-->
<html class="no-js" lang="en">
<!--<![endif]-->

<head>
    <!--- Basic Page Needs
    ================================================== -->
    <meta charset="utf-8">
    <title>Sebastian Zajac|homepage</title>
    <meta name="Description" content="Sebastian Zając's Home Page. Physics, Math &amp; More">
    <meta name="Keywords" content="Sebastian Zając,  Zając, Zajac, data scientists, statistics, physics, neutrino physics, neutrino, python, RNA, SGH, Research">
    <meta name="Author" content="Sebastian Zając">
    <meta name="robots" content="index,follow" />
    <!-- Mobile Specific Metas
   ================================================== -->
    <meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1">

    <!-- CSS
    ==================================================

In [39]:
# dane z internetu API
def pobierz_info_slowa(slowo):
    language = 'en-gb'
    headers = {"app_id":"c7f6d128",
             "app_key":"73ea2ed8109721300050137e74044fa6"}
    url_1 = "https://od-api.oxforddictionaries.com:443/api/v2/entries/"
    url = f"{url_1}{language}/{slowo.lower()}"
    return requests.get(url, headers=headers)

In [40]:
odp = pobierz_info_slowa('streaming')

In [43]:
[pobierz_info_slowa(x).json() for x in ["dog",'cat','add']]

[{'id': 'dog',
  'metadata': {'operation': 'retrieve',
   'provider': 'Oxford University Press',
   'schema': 'RetrieveEntry'},
  'results': [{'id': 'dog',
    'language': 'en-gb',
    'lexicalEntries': [{'derivatives': [{'id': 'doglike', 'text': 'doglike'}],
      'entries': [{'etymologies': ['Old English docga, of unknown origin'],
        'pronunciations': [{'audioFile': 'https://audio.oxforddictionaries.com/en/mp3/dog_gb_2.mp3',
          'dialects': ['British English'],
          'phoneticNotation': 'IPA',
          'phoneticSpelling': 'dɒɡ'}],
        'senses': [{'definitions': ['a domesticated carnivorous mammal that typically has a long snout, an acute sense of smell, non-retractable claws, and a barking, howling, or whining voice.'],
          'domainClasses': [{'id': 'mammal', 'text': 'Mammal'}],
          'id': 'm_en_gbus0287810.007',
          'notes': [{'text': 'Canis familiaris, family Canidae (the dog family); probably domesticated from the wolf in the Mesolithic period.

In [41]:
odp.json()

{'id': 'streaming',
 'metadata': {'operation': 'retrieve',
  'provider': 'Oxford University Press',
  'schema': 'RetrieveEntry'},
 'results': [{'id': 'streaming',
   'language': 'en-gb',
   'lexicalEntries': [{'entries': [{'grammaticalFeatures': [{'id': 'mass',
         'text': 'Mass',
         'type': 'Countability'}],
       'pronunciations': [{'audioFile': 'https://audio.oxforddictionaries.com/en/mp3/streaming_gb_1.mp3',
         'dialects': ['British English'],
         'phoneticNotation': 'IPA',
         'phoneticSpelling': 'ˈstriːmɪŋ'}],
       'senses': [{'definitions': ['a method of transmitting or receiving data (especially video and audio material) over a computer network as a steady, continuous flow, allowing playback to start while the rest of the data is still being received.'],
         'domainClasses': [{'id': 'audio', 'text': 'Audio'},
          {'id': 'computing', 'text': 'Computing'},
          {'id': 'video', 'text': 'Video'}],
         'id': 'm_en_gbus1001810.012',


In [ ]:
# obiektowość w bazach danych 
from sqlalchemy import create_engine
from sqlalchemy import Column, String, Integer
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

engine = create_engine('sqlite:///dane.db')
base = declarative_base()

In [ ]:
class Tabela1(base):
    """tabela danych"""
    __tablename__ = 'tablica'
    tablica_id = Column(Integer, primary_key=True)
    data = Column(String)
    tablica2_id = Column(Integer)
    cena = Column(Integer)
    
    def __init__(self, tablica_id, data, tablica2_id, cena):
        self.tablica_id = tablica_id
        self.data = data
        self.tablica2_id = tablica2_id
        self.cena = cena

In [ ]:
base.metadata.create_all(engine)

In [ ]:
from sqlalchemy.orm import sessionmaker # Stworzenie nowej sesji
Session = sessionmaker(bind=engine) 
session = Session()

In [ ]:
for t in range(10):
    dane = Tabela1(t, f'{t}/11/2021', t**2+t*2-2, 1000*t+500)
    session.add(dane)

In [ ]:
session.commit()

In [ ]:
for s in session.query(Tabela1).all():
    print(s.tablica_id, s.data, s.cena)

In [ ]:
# MLFlow

In [ ]:
!pip install mlflow --quiet

In [1]:
import mlflow
print(mlflow.__version__)

1.20.1


In [2]:
with mlflow.start_run(run_name= "first mlflow on jupyter"):
    mlflow.log_metric("m1",2.0)
    mlflow.log_param("p1","mlflow-notebook")

In [44]:
get_ipython().system_raw("mlflow ui --port 5000 &")

In [ ]:
!pip install sklearn xgboost 

In [5]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss
import xgboost as xgb
import mlflow
import mlflow.xgboost

iris = datasets.load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=24)

In [6]:
mlflow.xgboost.autolog()

In [7]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [8]:
with mlflow.start_run():
    params = {
        "objective":"multi:softprob",
        "num_class":3,
        "learning_rate":1,
        "eval_metric":"mlogloss",
        "seed": 24,
    }
    model = xgb.train(params, dtrain, evals=[(dtrain, "train")])
    y_prob = model.predict(dtest)
    y_pred = y_prob.argmax(axis=1)
    loss = log_loss(y_test, y_prob)
    acc = accuracy_score(y_test, y_pred)
    
    mlflow.log_metrics({"log_loss": loss, "accuracy":acc})
    

[0]	train-mlogloss:0.25530
[1]	train-mlogloss:0.12430
[2]	train-mlogloss:0.07452
[3]	train-mlogloss:0.04644
[4]	train-mlogloss:0.03723
[5]	train-mlogloss:0.03249
[6]	train-mlogloss:0.03015
[7]	train-mlogloss:0.02869
[8]	train-mlogloss:0.02689
[9]	train-mlogloss:0.02550


In [9]:
mlflow.get_tracking_uri()

'file:///Users/air/Desktop/mlruns'

In [10]:
mlflow.get_artifact_uri()

'file:///Users/air/Desktop/mlruns/0/ee1f4a1f78fa4f80bcd919d4dc9cc68b/artifacts'